## Let's segment some Stuff!


#### For a smooth ride, use a kernel from a nice clean virtualenv with (the right) TF and other dependencies installed: 

1. Edit ```requirements``` to have the right version of tensorflow[-gpu] ,
   <br>corresponding to your HW & CUDA setup (yep, it's not cross-compatible. Sad.)
2. Try the following recipe:
```
    mkdir ~/envs && mkdir ~/envs/segmentor
    virtualenv ~/envs/segmentor -p python2  # w.o. --system-packages. let's have it clean
    source ~/envs/segmentor/bin/activate
    which pip                      # verify it's the virtualenv's (not system's) pip
    cd seg-mentor                  # .where you cloned the repo
    pip install -r requirements
    which ipython                  # verify it's the virtualenv's (not system's) ipython
    ipython kernel install --user --name=segmentor
    deactivate                     # we'll want to run the system's generic jupyter
    jupyter notebook [--ip=* [--more-flags]]
```
3. Open the ```play-with-me.ipynb``` notebook in browser, then click ```Kernel-> Change kernel -> segmentor```
4. If stuff doesn't work, you're trying to be smart, or your platform is special in any other way,
   <br> Follow [THIS](http://anbasile.github.io/programming/2017/06/25/jupyter-venv/) and [THIS](https://www.tensorflow.org/install/install_linux) and figure it out:)

In [ ]:
import tensorflow as tf
print tf.__version__

#### Download a pre-trained model

If ```wget``` won't work, get it the GUI way (yikes) from [https://github.com/hailotech/seg-mentor/releases]

In [ ]:
!mkdir tmp   # convention is having pre-trained seg models as subdirs of <repo>/tmp/
!wget https://github.com/hailotech/seg-mentor/releases/download/v0.5/inception_v1_fcn16.tgz
!tar -xzvf inception_v1_fcn16.tgz -C tmp/

#### Download some media

In [ ]:
!wget https://github.com/hailotech/seg-mentor/releases/download/v0.5/perfectparking.mp4    
#  use browser if this doesn't work..

In [ ]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src="perfectparking.mp4" type="video/mp4">
</video></div>

#### Now let's prepare and run the model:

In [ ]:
from fcn_test import *
import json 

traindir = 'tmp/Apr24_inception_v1__1/'
checkpoint = traindir+'fcn.ckpt' 
pixels = 512
cfg = json.load(open(traindir+'/runargs'))

fcn_builder = fcn_arch.FcnArch(number_of_classes=number_of_classes, is_training=False, net=cfg['basenet'],
                               trainable_upsampling=cfg['trainable_upsampling'], fcn16=cfg['fcn16'])

def fcnfunc_img2labels(img):
    tmp = tf.argmax(fcn_builder.build_net(img), dimension=3)
    return tf.expand_dims(tmp, 3)

In [ ]:
tf.reset_default_graph()
segment_movie(fcnfunc_img2labels, checkpoint, 'perfectparking.mp4', pixels)

This should give you a gist of how the code's high level part works. You'll have to dive into the files for further details, I guess...
#### CLI note: same functionality available as:
``` source ~/envs/segmentor/bin/activate
python fcn_test.py --traindir=tmp/Apr24_inception_v1__1 --moviepath=perfectparking.mp4
```

#### OK let's see what we got:

In [ ]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src="perfectparking_segmented.mp4" type="video/mp4">
</video></div>

Well, you all probably want a smart API then movie.mp4 -> movie_segmented.mp4..
<br> I guess you'll need to dive a bit into the code for that. As russian proverb goes, "without work you can't even get a small fish out of the pond"

#### Let's segment a single image now, with legends and stuff:

In [ ]:
!wget https://c1.staticflickr.com/9/8525/8609166764_6ab4366126_b.jpg -O turkeycar.jpg

In [ ]:
from fcn_test import *
%matplotlib inline

tf.reset_default_graph()
segment_image(fcnfunc_img2labels, checkpoint, 'turkeycar.jpg', pixels)

#### hmm... in fact, in contrast to our training set, this is a high-res image, why not try and leverage that:

In [ ]:
tf.reset_default_graph()
segment_image(fcnfunc_img2labels, checkpoint, 'turkeycar.jpg', pixels=1024)

#### CLI note: same functionality available as:
``` source ~/envs/segmentor/bin/activate
python fcn_test.py --traindir=tmp/Apr24_inception_v1__1 --imagepath=turkeycar.jpg.mp4 --pixels=1024
```

### So, what about the actual validation set? 
#### well, let's run the full test, visualizing a few images along the way:

Note we here run the test at "native" resolution of the image, mainly to give nice pictures w.o. ugly margins.
<br>Another option is to rescale long side to *pixels* then pad to square,
<br>using either the values used at train-time (normally 512, usually yielding best mIoU) or a custom value. 

In [ ]:
tf.reset_default_graph()
iterator = get_data_feed(None)
image, annotation = iterator.get_next()
fcnfunc_img2labels = utils.inference.adapt_network_for_any_size_input(fcn_builder.build_net, 32)
predictions = fcnfunc_img2labels(tf.expand_dims(image, axis=0))
def viz_cb(i, (image_np, upsampled_predictions, annotation_np)):
    if 20 <= i < 25:
        visualize(image_np, annotation_np, upsampled_predictions, i)
scan_val_set(image, annotation, predictions, checkpoint, iterator,
             callback=viz_cb, more_tensors_to_eval=[image, annotation, predictions])

#### CLI note: same functionality available as:
``` source ~/envs/segmentor/bin/activate
python fcn_test.py --traindir=tmp/Apr24_inception_v1__1 --first2viz=10 --last2viz=15 --pixels=0
```

check out CLI help on resolution config via *pixels* arg